In [7]:
# Here, we are making a Movie-Recommendation System based only on Correlation.
# Correlation : It is a Statistical Measure to describe the extent to which two variables are linearly related.
# Generally,there are two types of Correlation. They are:
#     1. Positive Correlation: If both of the variables are moving in the same direction then the two variables are said to be positively Correlated.
#     2. Negative Correlation: If both of the variables are moving in different direction then they are said to be negatively correlated.
        

In [10]:
import sys  
!{sys.executable} -m pip install pandas

In [11]:
#Importing the required Libraries:
import pandas as pd
import numpy as np
import os
import warnings

In [12]:
warnings.filterwarnings('ignore') #To ignore warnings if generated

In [15]:
# Selecting the working directory and loading Dataset
working_dir = os.getcwd()
columns_name = ['user_id','item_id','rating','timestamp']
df = pd.read_csv(working_dir + '/u.data',sep = '\t',names = columns_name)
df.head(10) #Display first 10 rows of dataset

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [16]:
df.shape # Size of datset

(100000, 4)

In [17]:
df['user_id'].nunique() #Unique Values of users in data

943

In [18]:
df['item_id'].nunique() #Unique values of items in data

1682

In [20]:
#Importing second dataset:
movies_title = pd.read_csv(working_dir + '/u.item',sep = '\|',header = None)
movies_title.shape

(1682, 24)

In [21]:
movies_title.head()
#we need only movie name and item ID
#so,remove remaining columns
#first two columns
movies = movies_title[[0,1]]
movies.columns = ['item_id','title']
movies.tail()
df = pd.merge(df,movies,on = 'item_id')
df.tail()

,user_id,item_id,rating,timestamp,title
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."
99999,655,1641,3,887427810,Dadetown (1995)


In [22]:
#Finding average rating for a movie
ratings = pd.DataFrame(df.groupby('title').mean()['rating'])
ratings.head()
#Number of ratings
ratings['num_of_ratings'] = pd.DataFrame(df.groupby('title').count()['rating'])
ratings.head()

,rating,num_of_ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


In [23]:
movie_mat = df.pivot_table(index = "user_id",columns = "title",values = "rating")
movie_mat.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [24]:
#we make a generalised function for recommending
def predict_movies(movie_name):
  movie_user_ratings = movie_mat[movie_name]
  similar_to_movie = movie_mat.corrwith(movie_user_ratings)
  corr_movie = pd.DataFrame(similar_to_movie,columns = ['correlation'])
  corr_movie.dropna(inplace=True) #removing NaN values
  corr_movie = corr_movie.join(ratings['num_of_ratings'])
  #setting a threshold of 100 for no. of ratings
  predictions = corr_movie[corr_movie['num_of_ratings']>100].sort_values('correlation',ascending = False)
  return predictions

In [25]:
#Testing:
predict_my_movie = predict_movies('Absolute Power (1997)')
predict_my_movie.head(15)

,correlation,num_of_ratings
title,,
Absolute Power (1997),1.000000,127
Murder at 1600 (1997),0.727512,218
Volcano (1997),0.600681,219
"African Queen, The (1951)",0.586266,152
Mimic (1997),0.583524,101
Cool Hand Luke (1967),0.581955,164
Under Siege (1992),0.564634,124
Die Hard: With a Vengeance (1995),0.558460,151
Jungle2Jungle (1997),0.556533,132
